# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df = pd.read_csv("cities.csv")
df

Unnamed: 0    Lat     Lng  Max Temp  Humidity  Cloudiness  Wind Speed  \
0       jamestown  42.10  -79.24     16.67        87          90        5.10   
1         rikitea -23.12 -134.97     22.70        85           7        4.39   
2    oktyabrskoye  43.06   44.74     25.56        65          40        3.00   
3         shimoda  34.67  138.95     22.78        94          89        4.37   
4         isangel -19.55  169.27     22.03        83         100        5.81   
..            ...    ...     ...       ...       ...         ...         ...   
571       capreol  46.71  -80.92     17.78        93          75        6.70   
572      frontera  26.93 -101.45     22.78        88         100        0.45   
573        tiznit  29.58   -9.50     35.97        17           0        4.37   
574      harqalah  36.03   10.51     30.00        42          20        5.10   
575      sobolevo  54.43   31.90     17.26        87          98        0.48   

    Country        Date  
0        US  1599485295  
1        PF  1599485454  
2        RU  1599485454  
3        JP  1599485454  
4        VU  1599485455  
..      ...         ...  
571      CA  1599485626  
572      MX  1599485626  
573      MA  1599485571  
574      TN  1599485627  
575      RU  1599485627  

[576 rows x 9 columns]

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key) 

In [46]:
m = gmaps.Map()
locs = df[["Lat", "Lng"]].astype(float)
m.add_layer(gmaps.heatmap_layer(locs, weights=df.Humidity, max_intensity=100))
#commented out to prevent api key reveal on github
#m

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [31]:
#A max temperature lower than 80 degrees but higher than 70.
hotel_df = df.loc[df["Max Temp"] > 21,:]
hotel_df = df.loc[df["Max Temp"] < 32,:]

#Wind speed less than 10 mph.
hotel_df = df.loc[df["Wind Speed"] < 10,:]

#Zero cloudiness.
hotel_df = df.loc[df.Cloudiness == 0,:]

hotel_df.columns = ['City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness',
       'Wind Speed', 'Country', 'Date']
hotel_df

City    Lat     Lng  Max Temp  Humidity  Cloudiness  Wind Speed  \
8       vredendal -31.67   18.50     29.45        21           0        5.11   
10         albany  42.60  -73.97     20.00        69           0        0.89   
13      saskylakh  71.92  114.08      6.36        63           0        4.00   
18        ahipara -35.17  173.17     10.16        79           0        3.26   
21   khorramshahr  30.44   48.17     46.00        10           0        3.10   
..            ...    ...     ...       ...       ...         ...         ...   
556       humaita  -7.51  -63.02     37.23        18           0        1.85   
559        riyadh  24.69   46.72     39.00        13           0        2.60   
567         kidal  18.44    1.41     36.60        26           0        1.62   
570          asfi  32.30   -9.24     29.42        43           0        8.39   
573        tiznit  29.58   -9.50     35.97        17           0        4.37   

    Country        Date  
8        ZA  1599485456  
10       US  1599485304  
13       RU  1599485457  
18       NZ  1599485459  
21       IR  1599485460  
..      ...         ...  
556      BR  1599485563  
559      SA  1599485444  
567      ML  1599485596  
570      MA  1599485625  
573      MA  1599485571  

[106 rows x 9 columns]

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [32]:
import json
hotels = []
for i,x in hotel_df.iterrows():
    hot = requests.get(f"https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input=hotel&inputtype=textquery&locationbias=circle:5000@{x.Lat},{x.Lng}&key={g_key}").json()
    resp = requests.get(f"https://maps.googleapis.com/maps/api/place/details/json?key={g_key}&place_id={hot['candidates'][0]['place_id']}").json()
    hotels.append(resp['result']['name'])
    #print(resp['result']['geometry']['location'])
hotels

['Melkboomsdrift Guest House and Conference Centre',
 'The Century House, Ascend Hotel Collection',
 'Best Western Plus Irvine Spectrum Hotel',
 'Orana Motor Inn & Restaurant',
 'Hostel',
 'Sikakubete Guest House Kalabo Town Council',
 'Atlanticview Cape Town Boutique Hotel',
 'Hotel Hospedería Palacio de Allepuz',
 'Cossam Guest House',
 'Alrwasi Hotel',
 'Best Western Plus Irvine Spectrum Hotel',
 'Arniston Spa Hotel',
 'Birkenhead House',
 'Fendalton Park Luxury Accommodation',
 'Hotel Riviera',
 'Château de la Chaix',
 'Bam Tourist Hotel',
 'Diplomat Hotel',
 'Mövenpick Hotel Qassim',
 'Hotel Parque Serra da Lousã',
 'Hotel Alfil Express',
 'LÜDERITZ NEST HOTEL',
 'Alamdar Hotel',
 'Kenkiyak Khostel',
 'Ashraf Hotel',
 'هتل و رستوران قصر',
 'Shangri-La Hotel, Ulaanbaatar',
 'The Esplanade Hotel',
 'Springbok Inn by Country Hotels',
 'Buudai Hotel',
 'Bannisters Pavilion Rooftop Bar & Grill',
 'Guest House Sibirskaya',
 'Hotel Burdy',
 'Готель LO MO',
 'McMillans of Metung Coastal R

In [36]:
hotel_df.loc["Hotel Name"] = hotels
hotel_df

City    Lat     Lng  Max Temp  Humidity  Cloudiness  Wind Speed  \
8       vredendal -31.67   18.50     29.45        21           0        5.11   
10         albany  42.60  -73.97     20.00        69           0        0.89   
13      saskylakh  71.92  114.08      6.36        63           0        4.00   
18        ahipara -35.17  173.17     10.16        79           0        3.26   
21   khorramshahr  30.44   48.17     46.00        10           0        3.10   
..            ...    ...     ...       ...       ...         ...         ...   
556       humaita  -7.51  -63.02     37.23        18           0        1.85   
559        riyadh  24.69   46.72     39.00        13           0        2.60   
567         kidal  18.44    1.41     36.60        26           0        1.62   
570          asfi  32.30   -9.24     29.42        43           0        8.39   
573        tiznit  29.58   -9.50     35.97        17           0        4.37   

    Country        Date                                        Hotel Name  
8        ZA  1599485456  Melkboomsdrift Guest House and Conference Centre  
10       US  1599485304        The Century House, Ascend Hotel Collection  
13       RU  1599485457           Best Western Plus Irvine Spectrum Hotel  
18       NZ  1599485459                      Orana Motor Inn & Restaurant  
21       IR  1599485460                                            Hostel  
..      ...         ...                                               ...  
556      BR  1599485563                             Humaitá Quality Hotel  
559      SA  1599485444                        Hyatt Regency Riyadh Olaya  
567      ML  1599485596           Best Western Plus Irvine Spectrum Hotel  
570      MA  1599485625                      RIAD LE CHEVAL BLANC - HÔTEL  
573      MA  1599485571                                       Riad Janoub  

[106 rows x 10 columns]

In [38]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [45]:
# Add marker layer ontop of heat map
print(len(locations))

m.add_layer(gmaps.marker_layer(locations))

# Display figure
#commented out to prevent api key reveal on github
#m

106
